In [ ]:
!pip install selenium webdriver-manager

In [12]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import UnexpectedAlertPresentException
import re
import time

options = Options()
options.add_argument("user-data-dir=C:\\user-data\\kimfl")
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

driver = webdriver.Chrome(options=options)
driver.get("https://gcloud.csi.go.kr/cmq/main.do")



In [ ]:
for cookie in selenium_cookies:
    session.cookies.set(cookie['name'], cookie['value'])

detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
target_rqst_nos = ["RQ-2025-242229"] 
all_details = []

for rqst_no in target_rqst_nos:
    payload = {'rqltNo': rqst_no, 'pageMode': 'recv', 'searchKey': '1', 'searchDateKey': '1'}
    try:
        response = session.post(detail_url, data=payload, timeout=100)        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            row_data = {"의뢰번호": rqst_no}
            
            # [사진 54b423 분석 결과] th-td가 일렬로 나열된 모든 태그 수집
            all_tags = soup.find_all(['th', 'td'])
            
            # 수집 대상 키워드 (사진에 적힌 글자 그대로)
            targets = ["접수번호", "최종진행상태", "의뢰기관", "공사명", "발주자", "시공자"]
            
            for t in targets:
                row_data[t] = "" 
                for i, tag in enumerate(all_tags):
                    # 모든 공백과 특수문자를 제거하여 텍스트 순수 비교
                    clean_text = re.sub(r'\s+', '', tag.get_text())
                    
                    if tag.name == 'th' and t in clean_text:
                        # th를 찾으면 무조건 바로 다음(i+1) td가 정답 데이터임
                        if i + 1 < len(all_tags) and all_tags[i+1].name == 'td':
                            row_data[t] = all_tags[i+1].get_text(strip=True)
                            break

            # [사진 53ac9e 분석] 하단 처리자 추출
            row_data["특정처리자"] = ""
            hist_section = soup.find(id="rqst_hist_div")
            if hist_section:
                rows = hist_section.select("tbody tr")
                for r in rows:
                    cols = r.find_all("td")
                    if len(cols) >= 3 and "한국건설품질시험원" in cols[1].get_text():
                        row_data["특정처리자"] = cols[2].get_text(strip=True)

            all_details.append(row_data)
            print(f"[{rqst_no}] 추출 완료")
    except Exception as e:
        print(f"오류: {e}")

df = pd.DataFrame(all_details)
display(df)
# df.to_excel("final_refined_data.xlsx", index=False)

In [ ]:
detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
target_rqst_nos = ["RQ-2025-242229"] 
payload = {'rqltNo': rqst_no, 'pageMode': 'recv', 'searchKey': '1', 'searchDateKey': '1'}
response = session.post(detail_url, data=payload, timeout=100)
soup = BeautifulSoup(response.text, 'html.parser')
row_data = {"의뢰번호": rqst_no}
all_tags = soup.find_all(['th', 'td'])

<!-- print(all_tags) -->

In [8]:
# 1. 세션 생성
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
})

# 2. 로그인 수행 (CSI 로그인 주소와 필드명을 확인해야 합니다)
# 보통 /cmq/com/loginProc.do 형태일 가능성이 높습니다.
login_url = "https://gcloud.csi.go.kr/cmq/com/loginProc.do" 
login_payload = {
    'userId': 'youngjun',      # 실제 아이디
    'userPw': 'k*1800*92*' # 실제 비밀번호
}
session.post(login_url, data=login_payload)

# 3. 데이터 추출
detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
target_rqst_nos = ["RQ-2025-242229"]

for rqst_no in target_rqst_nos:
    payload = {'rqltNo': rqst_no, 'pageMode': 'recv', 'searchKey': '1', 'searchDateKey': '1'}
    response = session.post(detail_url, data=payload)
    
    # 텍스트가 깨진다면 euc-kr 또는 utf-8 시도
    response.encoding = 'utf-8' 
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 데이터 추출 로직
    row_data = {"의뢰번호": rqst_no}
    
    # 이제 th와 td가 있는지 확인합니다.
    tables = soup.find_all('table')
    if not tables:
        print(f"{rqst_no}: 여전히 데이터를 찾을 수 없습니다. (로그인 실패 혹은 다른 페이지 구조)")
        continue

    # 데이터 매핑 (기존 로직)
    ths = soup.find_all('th')
    tds = soup.find_all('td')
    for th, td in zip(ths, tds):
        row_data[th.get_text(strip=True)] = td.get_text(strip=True)
        
    print(row_data)

RQ-2025-242229: 여전히 데이터를 찾을 수 없습니다. (로그인 실패 혹은 다른 페이지 구조)


In [4]:
# [중요] 브라우저에서 복사한 'cookie:' 값을 여기에 붙여넣으세요.
user_cookie = "JSESSIONID=XQS5v4w_WDD5_T2S5CNhZzM-lzLn8nNQ5DRgppTM.cmq02; WMONID=NvX7vjvx50y"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Cookie': user_cookie,
    'Referer': 'https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvList.do',
    'Content-Type': 'application/x-www-form-urlencoded'
}

# 우리가 데이터를 가져올 상세 페이지 주소
detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"

# 요청할 데이터 (이미지의 의뢰번호 적용)
payload = {
    'rqltNo': 'RQ-2025-242229', 
    'pageMode': 'recv', 
    'searchKey': '1', 
    'searchDateKey': '1'
}

# 실행 및 결과 확인
try:
    response = requests.post(detail_url, data=payload, headers=headers, timeout=10)
    response.encoding = response.apparent_encoding # 자동 인코딩 설정
    
    if "의뢰" in response.text:
        print("✅ 성공: 상세 데이터를 성공적으로 불러왔습니다!")
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 테이블 데이터 파싱
        print("\n--- [조회 결과] ---")
        for row in soup.find_all('tr'):
            th = row.find('th')
            td = row.find('td')
            if th and td:
                print(f"{th.get_text(strip=True)} : {td.get_text(strip=True)}")
    else:
        print("❌ 실패: 데이터가 없습니다. 쿠키를 다시 확인하거나 로그아웃 후 다시 로그인해보세요.")
        
except Exception as e:
    print(f"오류 발생: {e}")

✅ 성공: 상세 데이터를 성공적으로 불러왔습니다!

--- [조회 결과] ---
반려 사유 : 
취소 사유 : 
요청자 : 이동규
요청일시 : 2025-12-24 17:47:51
요청 사유 : 
마감일시 : 
재공고 사유 : 
총 처리기간 : 선택123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149일(진행확정일은 산입하되, 토요일 및 공휴일은 제외)


In [4]:
print("1. 라이브러리 로드 완료")

# 설정 정보
user_cookie = "JSESSIONID=XQS5v4w_WDD5_T2S5CNhZzM-lzLn8nNQ5DRgppTM.cmq02; WMONID=NvX7vjvx50y"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Cookie': user_cookie,
    'Referer': 'https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvList.do'
}

def get_detail_data(rqst_no):
    detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
    payload = {
        'rqltNo': rqst_no, 
        'pageMode': 'recv', 
        'searchKey': '1', 
        'searchDateKey': '1'
    }
    
    print(f"2. {rqst_no} 서버 요청 중...")
    try:
        response = requests.post(detail_url, data=payload, headers=headers, timeout=10)
        print(f"3. 응답 코드: {response.status_code}") # 200이 나와야 정상
        
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        
        data = {"의뢰번호": rqst_no}
        rows = soup.find_all('tr')
        
        if not rows:
            print("⚠️ 데이터를 찾을 수 없습니다 (HTML 구조 확인 필요)")
            return None

        for row in rows:
            th = row.find('th')
            td = row.find('td')
            if th and td:
                key = th.get_text(strip=True)
                val = td.get_text(strip=True)
                if "선택123" in val:
                    val = val.split('일')[0].strip().split(' ')[-1] + "일"
                if key:
                    data[key] = val
        return data
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

target_list = ["RQ-2025-242229"]
final_results = []

for no in target_list:
    detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
    payload = {
        'rqltNo': no, 
        'pageMode': 'recv', 
        'searchKey': '1', 
        'searchDateKey': '1'
    }
    
    print(f"📡 {no} 본문 분석 시작...")
    response = requests.post(detail_url, data=payload, headers=headers)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # [중요] 페이지에 있는 모든 테이블을 강제로 다 긁어옵니다.
    print("\n=== 발견된 모든 데이터 항목 목록 ===")
    
    found_data = {"의뢰번호": no}
    tables = soup.find_all('table')
    
    for i, table in enumerate(tables):
        rows = table.find_all('tr')
        for row in rows:
            # th/td 구조가 아니더라도 일단 글자가 있으면 다 가져옴
            cells = row.find_all(['th', 'td'])
            if len(cells) >= 2:
                key = cells[0].get_text(strip=True)
                # 만약 항목명이 너무 길면 정리
                val = " ".join([c.get_text(strip=True) for c in cells[1:]])
                
                # 불필요한 숫자 나열 제거 로직
                if "선택123" in val:
                    val = val.split('일')[0].strip().split(' ')[-1] + "일"
                
                if key:
                    found_data[key] = val
                    print(f"[{key}] : {val[:50]}...") # 화면에 출력해서 확인

    final_results.append(found_data)

# 엑셀 저장
if final_results:
    df = pd.DataFrame(final_results)
    df.to_csv("CSI_조회결과_전체.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ 분석 완료! 총 {len(tables)}개의 테이블을 찾았습니다.")
    print("화면에 출력된 내용 중에 원하시는 데이터(공사명 등)가 보이시나요?")
    display(df.head())

1. 라이브러리 로드 완료
📡 RQ-2025-242229 본문 분석 시작...

=== 발견된 모든 데이터 항목 목록 ===
[처리유형] : 처리기관 처리자 처리일시 처리내용...
[저장] : 한솔종합건설 장성보 2025-12-24 15:37:32 ...
[의뢰신청] : 한솔종합건설 장성보 2025-12-24 15:37:38 ...
[의뢰서제출] : (주)신한종합건축사사무소 홍흥기 2025-12-24 16:02:27 의뢰확인 후 자동 제출...
[시험·검사 진행 확정] : 한국건설품질시험원 주식회사 이동규 2025-12-24 17:47:51 ...
[반려 사유] : ...
[취소 사유] : ...
[요청자] : 이동규...
[요청일시] : 2025-12-24 17:47:51...
[요청 사유] : ...
[마감일시] : ...
[재공고 사유] : ...
[총 처리기간] : 선택123456789101112131415161718192021222324252627282...

✅ 분석 완료! 총 7개의 테이블을 찾았습니다.
화면에 출력된 내용 중에 원하시는 데이터(공사명 등)가 보이시나요?


,의뢰번호,처리유형,저장,의뢰신청,의뢰서제출,시험·검사 진행 확정,반려 사유,취소 사유,요청자,요청일시,요청 사유,마감일시,재공고 사유,총 처리기간
0,RQ-2025-242229,처리기관 처리자 처리일시 처리내용,한솔종합건설 장성보 2025-12-24 15:37:32,한솔종합건설 장성보 2025-12-24 15:37:38,(주)신한종합건축사사무소 홍흥기 2025-12-24 16:02:27 의뢰확인 후 자...,한국건설품질시험원 주식회사 이동규 2025-12-24 17:47:51,,,이동규,2025-12-24 17:47:51,,,,선택12345678910111213141516171819202122232425262...


In [11]:
# [중요] 방금 다시 로그인해서 복사한 '최신' 쿠키를 넣으세요.
user_cookie = "JSESSIONID=9NMurfRvV3uS6WpFCgq9bljtWkWd3eNbSfbwIrNv.cmq02; WMONID=NvX7vjvx50y" 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Cookie': user_cookie,
    'Origin': 'https://gcloud.csi.go.kr',
    'Referer': 'https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvList.do',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest'
}

def get_data_by_search(rqst_no):
    # STEP 1: 목록에서 해당 번호가 있는지 먼저 검색 (image_96962b.jpg 구조 참고)
    list_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvList.do"
    search_payload = {
        'searchVal': rqst_no,  # 검색창에 입력할 번호
        'searchKey': '1',      # 의뢰번호 조건 선택
        'pageIndex': '1'
    }
    
    list_res = requests.post(list_url, data=search_payload, headers=headers)
    
    if rqst_no not in list_res.text:
        return f"❌ 목록에서 {rqst_no}를 찾을 수 없습니다. (권한이나 검색조건 확인 필요)"

    # STEP 2: 상세 데이터(Ajax) 가져오기 (image_9619f1.png 구조 참고)
    ajax_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/selectRqstViewAjax.do"
    ajax_payload = {
        'rqltNo': rqst_no,
        'pageMode': 'recv'
    }
    
    try:
        response = requests.post(ajax_url, data=ajax_payload, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 공사명 (bizNm) 추출
        biz_nm_tag = soup.find('td', id='bizNm')
        project_name = biz_nm_tag.get_text(strip=True) if biz_nm_tag else "공사명 없음"
        
        # 기타 왼쪽 표의 모든 데이터 수집
        detail_data = {"의뢰번호": rqst_no, "공사명": project_name}
        for tr in soup.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')
            if th and td:
                key = th.get_text(strip=True)
                val = td.get_text(strip=True)
                if key and key not in detail_data:
                    detail_data[key] = val
        
        return detail_data

    except Exception as e:
        return f"❌ 상세 데이터 추출 중 에러: {e}"

# 실행
target_no = "RQ-2025-242229"
result = get_data_by_search(target_no)

if isinstance(result, dict):
    print(f"✅ 성공: {result['공사명']}")
    df = pd.DataFrame([result])
    display(df)
else:
    print(result)

✅ 성공: 공사명 없음


,의뢰번호,공사명
0,RQ-2025-242229,공사명 없음
